In [2]:
from googletrans import Translator
import pandas as pd
import numpy as np
import time
import re
import pickle

In [3]:
df_kospi_filing_2011_2015 = pd.read_excel('C:\\Users\\sylim2357\\Documents\\SNU\\FVID\\10k delta\\Data\\kospi200_2011_2015.xlsx')

In [21]:
missing_list = [7,12,13,14,19,20,27,56,99,100,101,102,103,104,105,106,108,109,111,112,114,115,117,119,120,121,122,127,131,136,137,139,141,173,198,199,203,206,208,211,212,218,220,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,369,370,372,373,374,375]

In [ ]:
def make_sublist(text):
    batch = []
    batch_size = 5
    text_list = [text for text in re.split('\. ', text) if (len(text)>3)]
    num_batches = int(np.ceil(len(text_list)/batch_size))
    for i in range(num_batches):
        batch.append('. '.join(text_list[batch_size*i:batch_size*(i+1)]))
    return batch

text_series = df_kospi_filing_2011_2015['Text'].replace("\\n|전자공시시스템 dart\.fss\.or\.kr(\nPage \d{1,2})?(?=\s)|\s+", " ", regex=True)
text_series_sublists = text_series.apply(make_sublist)
trans_error = []
translated_series = []
for idx, text_sublist in enumerate(text_series_sublists):
    try:
        time.sleep(3)
        print(str(idx)+' being translated')
        translated = Translator().translate(text_sublist)
        print(str(idx)+' has been translated')
        translated_series.append('. '.join([trans.text for trans in translated]))
        print(str(idx)+' has been recorded')
    except:
        try:
            text_sublist_sublist = []
            print(str(idx)+' being translated again')
            for i in range(int(np.ceil(len(text_sublist)/batch_size))):
                time.sleep(3)
                translated = Translator().translate(text_sublist[batch_size*i:batch_size*(i+1)])
                text_sublist_sublist.append('. '.join([trans.text for trans in translated]))
            print(str(idx)+' has been translated')
            translated_series.append('. '.join(text_sublist_sublist))
            print(str(idx)+' has been recorded')
        except:
            print(str(idx)+' has error')
            trans_error.append(idx)
            translated_series.append(np.nan)
    if idx % 100 == 0:
            with open('translated.pickle', 'wb') as f:
                pickle.dump(translated_series, f)
            with open('trans_error.pickle', 'wb') as f:
                pickle.dump(trans_error, f)

0 being translated
0 has been translated
0 has been recorded
1 being translated
1 has been translated
1 has been recorded
2 being translated
2 being translated again
2 has error
3 being translated
3 has been translated
3 has been recorded
4 being translated
4 has been translated
4 has been recorded
5 being translated
5 has been translated
5 has been recorded
6 being translated
6 has been translated
6 has been recorded
7 being translated
7 being translated again
7 has error
8 being translated
8 being translated again
8 has error
9 being translated
9 has been translated
9 has been recorded
10 being translated
10 has been translated
10 has been recorded
11 being translated
11 being translated again
11 has error
12 being translated
12 being translated again
12 has error
13 being translated
13 being translated again


In [4]:
def make_sublist(text):
    batch = []
    batch_size = 5
    text_list = [text for text in re.split('\. |Page', text) if (len(text)>3) & ('dart.fss.or.kr' not in text)]
    num_batches = int(np.ceil(len(text_list)/batch_size))
    for i in range(num_batches):
        batch.append('. '.join(text_list[batch_size*i:batch_size*(i+1)]))
    return batch
text_sublist = make_sublist(df_kospi_filing_2011_2015.iloc[7].replace("\s\s+|\\n", " ", regex=True).Text)

In [6]:
# text_sublist = make_sublist(df_kospi_filing_2011_2015['Text'].replace("\s\s+|\\n", " ", regex=True).iloc[7])
translated_series = []
text_sublist_sublist = []
batch_size = 1
for i in range(int(np.ceil(len(text_sublist)/batch_size))):
    time.sleep(3)
    print(i)
    translated = Translator().translate(text_sublist[batch_size*i:batch_size*(i+1)])
    text_sublist_sublist.append('. '.join([trans.text for trans in translated]))
translated_series.append('. '.join(text_sublist_sublist))

0


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [71]:
len(translated_series)

1